# DataFrame com Spark
---

## 1. Bibliotecas

In [12]:
# Instalando os pacotes necessários

#!conda install pyspark -y
#!conda install -c conda-forge findspark -y

In [15]:
import findspark
findspark.find()
findspark.init()
import pyspark

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [7]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.getOrCreate()

ModuleNotFoundError: No module named 'pyspark.sql'